## Topic Modelling for News Articles

In [1]:
#pip install gensim
#pip install pyLDAvis

In [2]:

#Initial Imports
import pandas as pd
import nltk as nltk
import numpy as np

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

nltk.download('stopwords')

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aerku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\aerku\AppData\Local\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\aerku\AppData\Local\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [3]:
# Getting all the news articles 
articles = pd.read_csv('articles.csv', encoding="utf-8-sig")

In [4]:
articles.head()

,date,title,text,description
0,2020-12-31,$1B Intel investor says chipmaker must try to ...,An Intel investor with a billion dollar stake ...,An Intel investor with a billion dollar stake ...
1,2020-12-31,"The Morning After: Intel, AMD and Apple made 2...",A former Songkick employee shared his login in...,"Very, very soon, 2020 will be over. Did it tak..."
2,2020-12-31,Some Mac software has made it all the way from...,Mac developers are currently in the midst of a...,Mac developers are currently in the midst of a...
3,2020-12-31,Some Mac software has made it all the way from...,Mac developers are currently in the midst of a...,Mac developers are currently in the midst of a...
4,2020-12-31,"How to revive and restore M1 Macs, what the di...","Along with the shift to Apple Silicon, perform...","Along with the shift to Apple Silicon, perform..."


In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6639 entries, 0 to 6638
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         6639 non-null   object
 1   title        6639 non-null   object
 2   text         6639 non-null   object
 3   description  6618 non-null   object
dtypes: object(4)
memory usage: 207.6+ KB


In [6]:
# Cleaning the data by droping the null values
articles.dropna(inplace=True)

In [7]:
# Clean the data 

# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
stop = stopwords.words('english')

# Expand the default stopwords list if necessary
stop.extend(["going", "still", "last", "think", "see", "way", "n't", "well", "know", "much", "’", "top", "app", "/li", "li", "...", "'s", "_", "--", "one", "said", "td", "gb", "tr"])


#Set tokenization function
def clean_text(text):
    
    words = word_tokenize(text)
    
    words = list(filter(lambda w:w.lower(), words))
 
    words = list(filter(lambda t:t not in punctuation, words))
    
    words = list(filter(lambda t: t.lower() not in stop, words))
    
    token = [lemmatizer.lemmatize(word) for word in words]
    
    return token

In [8]:
# Applying the clean text function to the text
data_words = articles['text'].apply(clean_text)

data_words[0][:20]

['Intel',
 'investor',
 'billion',
 'dollar',
 'stake',
 'chipmaker',
 'say',
 'need',
 'make',
 'drastic',
 'change',
 'address',
 'changed',
 'fortune',
 'including',
 'attempting',
 'win',
 'back',
 'Apple',
 'client']

In [9]:
# Defining Gensim word simple preprocess 

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)
#data_words = gen_words(dwords)


In [10]:

id2word = corpora.Dictionary(data_words)


corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print(corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)


#from gensim.models import TfidfModel
#tfidf = TfidfModel(corpus, id2word=id2word)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 6), (15, 2), (16, 1), (17, 2), (18, 2), (19, 2)]
10-figure


In [11]:
# LDA Model

lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, 
                                           id2word = id2word, 
                                           num_topics= 7, 
                                           random_state = 100,
                                           update_every=1, 
                                           alpha = 'auto')

In [12]:
#Topics
print(lda_model.print_topics())


[(0, '0.042*"Apple" + 0.025*"Qualcomm" + 0.014*"iPhone" + 0.014*"Intel" + 0.014*"5G" + 0.013*"modem" + 0.013*"chip" + 0.009*"company" + 0.008*"year" + 0.007*"patent"'), (1, '0.031*"Apple" + 0.012*"new" + 0.009*"Mac" + 0.006*"year" + 0.006*"iPhone" + 0.005*"iPad" + 0.005*"Pro" + 0.005*"``" + 0.005*"\'\'" + 0.005*"also"'), (2, '0.025*"Visa" + 0.012*"/td" + 0.009*"Mastercard" + 0.009*"Apple" + 0.007*"Pro" + 0.006*"year" + 0.006*"Mac" + 0.005*"new" + 0.004*"iMac" + 0.004*"Intel"'), (3, '0.036*"Amazon" + 0.024*"price" + 0.023*"list" + 0.018*"deal" + 0.017*"normally" + 0.011*"Walmart" + 0.008*"code" + 0.008*"RAM" + 0.008*"Apple" + 0.007*"use"'), (4, '0.008*"company" + 0.006*"Apple" + 0.006*"Huawei" + 0.006*"like" + 0.005*"also" + 0.005*"device" + 0.005*"Google" + 0.005*"year" + 0.004*"new" + 0.004*"laptop"'), (5, '0.018*"Pro" + 0.015*"MacBook" + 0.014*"Apple" + 0.012*"laptop" + 0.011*"Intel" + 0.009*"model" + 0.008*"new" + 0.008*"Core" + 0.007*"processor" + 0.006*"display"'), (6, '0.012*"com

# Visualization of LDA model

In [21]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.106333  0.003190       1        1  20.708641
6      0.034841  0.132803       2        1  20.267038
5     -0.044323 -0.134831       3        1  19.129950
0     -0.101309  0.141936       4        1  14.271645
4     -0.030981  0.023538       5        1  12.515688
3      0.179932 -0.203866       6        1   9.629456
2      0.068172  0.037230       7        1   3.477582, topic_info=            Term          Freq         Total Category  logprob  loglift
206       Amazon  12078.000000  12078.000000  Default  30.0000  30.0000
1802    Qualcomm  11287.000000  11287.000000  Default  29.0000  29.0000
18177       Visa   5587.000000   5587.000000  Default  28.0000  28.0000
14         Apple  51109.000000  51109.000000  Default  27.0000  27.0000
731         list   7402.000000   7402.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
178         time    265.792608   8034.203664   Topic7  -5.8985  -0.0499
1843        deal    274.106719  10528.670501   Topic7  -5.8677  -0.2895
434           ''    269.392676  11659.600395   Topic7  -5.8851  -0.4089
787    processor    253.087064   9338.855938   Topic7  -5.9475  -0.2494
125         like    250.566663  10003.589212   Topic7  -5.9575  -0.3281

[636 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
434        1  0.240489            ''
434        2  0.245549            ''
434        3  0.114841            ''
434        4  0.213472            ''
434        5  0.060722            ''
...      ...       ...           ...
16734      5  0.021818  year-on-year
16734      6  0.021818  year-on-year
16734      7  0.545449  year-on-year
21834      5  0.968040          yoga
5937       1  0.954107             ★

[2474 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 1, 5, 4, 3])